# 🔍 Grover's Algorithm for Symmetric Key Attacks

**Author:** Mauro Risonho de Paula Assumpção aka firebitsbr  
**License:** MIT  
**Date:** August 7, 2025

## 🎯 Overview

Grover's algorithm provides a quadratic speedup for brute-force attacks against symmetric encryption, effectively halving the security level of symmetric keys.

### 🎯 Attack Scenarios:
- 🔑 **AES key recovery** (256-bit → 128-bit effective security)
- #️⃣ **Hash function preimage** attacks
- 🔐 **Password cracking** acceleration
- 🎲 **Cryptographic nonce** prediction

### ⚠️ **Legal Disclaimer**

This notebook is for **authorized security testing and educational purposes only**.

---

## 🐳 Docker Environment Configuration

This notebook connects to two Docker containers:
1. **houdinis_framework** - Quantum attack framework with CUDA/GPU support
2. **houdinis_target** - Vulnerable services (SSH, HTTPS with weak crypto)

Both containers run in an isolated network for safe testing.

In [ ]:
# 🐳 Docker Container Connection Setup
import subprocess
import json
import time
import socket

def check_docker_containers():
    """Check if Houdinis Docker containers are running"""
    try:
        result = subprocess.run(
            ['docker', 'ps', '--filter', 'name=houdinis', '--format', '{{json .}}'],
            capture_output=True, text=True, check=True
        )
        
        containers = []
        for line in result.stdout.strip().split('\n'):
            if line:
                containers.append(json.loads(line))
        
        return containers
    except Exception as e:
        print(f"❌ Error checking containers: {e}")
        return []

def start_containers():
    """Start Houdinis Docker containers if not running"""
    try:
        print("🚀 Starting Docker containers...")
        subprocess.run(
            ['docker', 'compose', '-f', '../docker/docker-compose.yml', 'up', '-d'],
            check=True, capture_output=True
        )
        time.sleep(5)  # Wait for containers to be ready
        print("✅ Containers started successfully")
    except Exception as e:
        print(f"❌ Error starting containers: {e}")

def get_container_ip(container_name):
    """Get IP address of a Docker container"""
    try:
        result = subprocess.run(
            ['docker', 'inspect', '-f', '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}', container_name],
            capture_output=True, text=True, check=True
        )
        return result.stdout.strip()
    except Exception as e:
        print(f"❌ Error getting IP for {container_name}: {e}")
        return None

# Check container status
print("🔍 Checking Docker containers...")
containers = check_docker_containers()

if len(containers) < 2:
    print("⚠️  Containers not running. Starting them...")
    start_containers()
    containers = check_docker_containers()

# Display container status
print("\n📦 Container Status:")
print("=" * 60)
for container in containers:
    print(f"  🐳 {container['Names']}: {container['Status']}")
    print(f"     Image: {container['Image']}")
    print(f"     Ports: {container.get('Ports', 'N/A')}")

# Get container IPs
TARGET_IP = get_container_ip('houdinis_target')
FRAMEWORK_IP = get_container_ip('houdinis_framework')

print("\n🌐 Network Configuration:")
print("=" * 60)
print(f"  🎯 Target IP: {TARGET_IP}")
print(f"  🔧 Framework IP: {FRAMEWORK_IP}")
print("\n✅ Docker environment ready!")
print("=" * 60)

In [ ]:
# 🎯 Docker Execution Helper Functions
from typing import Dict, Any

def exec_in_houdinis(command: str, workdir: str = "/app") -> Dict[str, Any]:
    """Execute command in houdinis_framework container"""
    try:
        cmd = ['docker', 'exec', '-w', workdir, 'houdinis_framework', 'bash', '-c', command]
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        return {
            'stdout': result.stdout,
            'stderr': result.stderr,
            'returncode': result.returncode
        }
    except subprocess.TimeoutExpired:
        return {'stdout': '', 'stderr': 'Command timeout', 'returncode': -1}
    except Exception as e:
        return {'stdout': '', 'stderr': str(e), 'returncode': -1}

def test_target_service(service: str = 'ssh', port: int = 22) -> bool:
    """Test if target service is reachable"""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(3)
        result = sock.connect_ex((TARGET_IP, port))
        sock.close()
        return result == 0
    except:
        return False

# Test connectivity to target
print("🔍 Testing target services...")
print("=" * 60)

services = [
    ('SSH', 22),
    ('HTTP', 80),
    ('HTTPS', 443)
]

for service_name, port in services:
    # Check from host
    reachable = test_target_service(service_name.lower(), port)
    status = "✅ Reachable" if reachable else "❌ Not reachable"
    print(f"  {service_name} (port {port}): {status}")

print("\n🧪 Testing Houdinis framework in container...")
# Test if quantum modules are available in container
result = exec_in_houdinis("python3 -c 'import sys; sys.path.insert(0, \"/app\"); from quantum.backend import QuantumBackendManager; print(\"Houdinis OK\")'")
if result['returncode'] == 0 and 'OK' in result['stdout']:
    print("  ✅ Houdinis framework is functional")
else:
    print(f"  ⚠️  Note: {result['stderr'] if result['stderr'] else 'Container may need dependencies'}")

print("\n✅ Connectivity tests complete!")
print("=" * 60)

In [ ]:
# 🚀 Execute Grover Attack Against Target
from typing import Optional

def run_grover_attack_via_docker(
    target_host: str = 'target', 
    key_length: int = 4, 
    method: str = 'aes'
) -> Dict[str, Any]:
    """
    Execute Grover's algorithm attack using Houdinis container
    
    Args:
        target_host: Target hostname (default: 'target' - the vulnerable container)
        key_length: Key length in bits for demonstration (4-8 bits)
        method: Attack method ('aes', 'hash', 'password')
    
    Returns:
        Dict with stdout, stderr, and returncode
    """
    print(f"🎯 Launching Grover's Algorithm Attack")
    print("=" * 60)
    print(f"  Target: {target_host}")
    print(f"  Method: {method}")
    print(f"  Key Length: {key_length} bits")
    print("=" * 60)
    
    # Create attack command
    if method == 'aes':
        # Use Houdinis' Grover bruteforce module
        # Note: These imports work inside the Docker container
        attack_cmd = f"""
python3 << 'EOF'
import sys
sys.path.insert(0, '/app')

# These imports work inside the container (not on host)
from exploits.grover_bruteforce import GroverBruteforceExploit  # type: ignore
from quantum.backend import QuantumBackendManager  # type: ignore

# Initialize backend
backend_mgr = QuantumBackendManager()
backend = backend_mgr.get_backend('aer_simulator')

# Create exploit
exploit = GroverBruteforceExploit(
    backend=backend,
    key_length={key_length},
    target_function='aes'
)

# Run attack
print("\\n🔐 Starting Grover attack simulation...")
result = exploit.run()

print(f"\\n📊 Attack Results:")
print(f"  Found Key: {{result.get('key', 'N/A')}}")
print(f"  Iterations: {{result.get('iterations', 'N/A')}}")
print(f"  Success Rate: {{result.get('success_rate', 0):.2%}}")
print(f"  Quantum Speedup: {{result.get('speedup', 1):.2f}}x")
EOF
"""
    else:
        attack_cmd = f"""
python3 -c "print('🔍 Grover attack simulation for {method} not yet implemented')"
"""
    
    # Execute in container
    print("\n⚙️  Executing attack in Houdinis container...")
    result = exec_in_houdinis(attack_cmd)
    
    if result['returncode'] == 0:
        print(result['stdout'])
        print("\n✅ Attack completed successfully!")
    else:
        print(f"❌ Attack failed:")
        print(f"  Error: {result['stderr']}")
    
    print("=" * 60)
    return result

# Example: Run a small-scale Grover attack demonstration
print("💡 Tip: Start with key_length=4 for fast demonstration")
print("   Larger keys (8-16 bits) will take longer but show realistic speedup\n")

In [ ]:
# 🎯 Attack SSH Service on Target Container
def attack_ssh_password():
    """Use Grover to accelerate SSH password cracking"""
    print("🔐 SSH Password Attack using Grover's Algorithm")
    print("=" * 60)
    
    # We know the target has password 'vulnerable' - let's simulate finding it
    attack_cmd = """
python3 << 'EOF'
import sys
import hashlib
sys.path.insert(0, '/app')

# Simulate Grover search for SSH password
target_hash = hashlib.sha256(b'vulnerable').hexdigest()
print(f"🎯 Target: SSH on 'target' container")
print(f"🔍 Known password hash: {target_hash[:16]}...")
print(f"\\n⚙️  Simulating Grover search in password space...")
print(f"   Classical search: ~2^32 attempts")
print(f"   Grover speedup: ~2^16 attempts")
print(f"   Speedup factor: 65536x faster!")
print(f"\\n✅ Password found: 'vulnerable'")
print(f"\\n🔓 Connection test:")

# Try to connect to target SSH
import paramiko
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
try:
    ssh.connect('target', username='root', password='vulnerable', timeout=5)
    stdin, stdout, stderr = ssh.exec_command('whoami')
    result = stdout.read().decode().strip()
    print(f"   ✅ SSH login successful as: {result}")
    ssh.close()
except Exception as e:
    print(f"   ❌ SSH connection failed: {e}")
EOF
"""
    
    result = exec_in_houdinis(attack_cmd)
    print(result['stdout'])
    if result['stderr']:
        print(f"Warnings: {result['stderr']}")
    print("=" * 60)
    return result

# Example usage (uncomment to run)
# attack_ssh_password()

In [ ]:
# 🔒 Attack HTTPS/TLS on Target Container
def attack_tls_rsa():
    """Demonstrate TLS RSA vulnerability to quantum attacks"""
    print("🔐 TLS/HTTPS RSA Certificate Attack")
    print("=" * 60)
    
    attack_cmd = """
python3 << 'EOF'
import sys
import ssl
import socket
sys.path.insert(0, '/app')

print("🎯 Target: HTTPS service on 'target' container")
print("🔍 Analyzing TLS certificate...")

try:
    # Connect to HTTPS service
    context = ssl.create_default_context()
    context.check_hostname = False
    context.verify_mode = ssl.CERT_NONE
    
    with socket.create_connection(('target', 443), timeout=5) as sock:
        with context.wrap_socket(sock, server_hostname='target') as ssock:
            cert = ssock.getpeercert_bin()
            cipher = ssock.cipher()
            version = ssock.version()
            
            print(f"\\n📜 Certificate Information:")
            print(f"   TLS Version: {version}")
            print(f"   Cipher Suite: {cipher[0]}")
            print(f"   Key Exchange: {cipher[1]}")
            print(f"   Encryption: {cipher[2]} bits")
            
            # Check if vulnerable to Shor's algorithm
            if 'RSA' in cipher[1]:
                print(f"\\n⚠️  VULNERABLE to Shor's Algorithm!")
                print(f"   RSA key can be factored using quantum computer")
                print(f"   Classical time: 2^2048 operations")
                print(f"   Quantum time: ~polynomial time")
                print(f"   Recommendation: Migrate to post-quantum crypto (e.g., CRYSTALS-Kyber)")
            
            print(f"\\n✅ TLS analysis complete")
            
except Exception as e:
    print(f"❌ Connection error: {e}")
    print(f"💡 Make sure target container is running")
EOF
"""
    
    result = exec_in_houdinis(attack_cmd)
    print(result['stdout'])
    if result['stderr']:
        print(f"Warnings: {result['stderr']}")
    print("=" * 60)
    return result

# Example usage (uncomment to run)
# attack_tls_rsa()

In [ ]:
# 🎬 Execute Complete Attack Demonstration
def run_full_attack_demo():
    """Execute all attacks against target container"""
    print("🚀 HOUDINIS QUANTUM ATTACK DEMONSTRATION")
    print("=" * 80)
    print("Target: houdinis_target container")
    print("Attacker: houdinis_framework container")
    print("=" * 80)
    
    attacks = [
        ("1️⃣  Grover's Algorithm - AES Key Recovery", 
         lambda: run_grover_attack_via_docker(key_length=6, method='aes')),
        ("2️⃣  SSH Password Cracking with Quantum Speedup", 
         attack_ssh_password),
        ("3️⃣  TLS/HTTPS RSA Certificate Analysis", 
         attack_tls_rsa),
    ]
    
    results = {}
    for name, attack_func in attacks:
        print(f"\n\n{'='*80}")
        print(f"🎯 {name}")
        print(f"{'='*80}\n")
        
        try:
            result = attack_func()
            results[name] = "✅ Success"
        except Exception as e:
            results[name] = f"❌ Failed: {e}"
            print(f"Error: {e}")
        
        time.sleep(1)
    
    # Summary
    print(f"\n\n{'='*80}")
    print("📊 ATTACK SUMMARY")
    print(f"{'='*80}")
    for attack, status in results.items():
        print(f"  {attack}: {status}")
    print(f"{'='*80}\n")
    
    return results

# Uncomment to run full demonstration:
# run_full_attack_demo()

---

## 📋 How to Use This Notebook

### Step 1: Start Docker Containers
Run cell 2 to check and start the Docker containers if needed.

### Step 2: Verify Connectivity
Run cell 3 to test connectivity to target services (SSH, HTTP, HTTPS).

### Step 3: Run Individual Attacks
Execute cells 5-7 for specific attacks:
- **Cell 5**: Grover's algorithm for AES key recovery
- **Cell 6**: SSH password cracking demonstration
- **Cell 7**: TLS/RSA certificate vulnerability analysis

### Step 4: Run Complete Demo
Run cell 8 to execute all attacks sequentially.

### 🎯 Attack Flow:
```
Jupyter Notebook → Docker Exec → houdinis_framework container
                                        ↓
                                  Quantum Attack
                                        ↓
                                houdinis_target container
                                  (Vulnerable Services)
```

### ⚠️ Important Notes:
- Containers must be running before executing attacks
- First cell execution may take longer (importing libraries)
- All attacks are contained within isolated Docker network
- No external network access from containers

---

In [ ]:
# 🛠️ Environment Setup and Imports
# Note: This notebook runs attacks inside Docker containers
# Quantum libraries (Qiskit) are available inside houdinis_framework container

import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Core imports (available on host)
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import random
import hashlib
import itertools
import string

# Quantum computing imports - only needed inside Docker container
# These will be imported when executing code via docker exec
QISKIT_AVAILABLE = False
try:
    from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
    from qiskit import transpile, Aer
    from qiskit.visualization import plot_histogram
    QISKIT_AVAILABLE = True
    print("✅ Qiskit imported successfully (running on host)")
except ImportError:
    print("ℹ️  Qiskit not available on host (will use Docker container)")

# Houdinis framework imports - only available inside container
HOUDINIS_AVAILABLE = False
try:
    # Add Houdinis to path if running locally
    houdinis_path = '/home/test/Downloads/github/portifolio/Houdinis'
    if os.path.exists(houdinis_path) and houdinis_path not in sys.path:
        sys.path.insert(0, houdinis_path)
    
    from quantum.backend import QuantumBackendManager
    from exploits.grover_bruteforce import GroverBruteforceExploit
    HOUDINIS_AVAILABLE = True
    print("✅ Houdinis modules imported successfully (running on host)")
except (ImportError, ModuleNotFoundError) as e:
    print("ℹ️  Houdinis modules not available on host (will use Docker container)")

# Configuration
plt.style.use('dark_background')

print("\n🚀 Environment setup complete!")
print(f"📅 Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐳 Docker execution mode: {'No' if (QISKIT_AVAILABLE and HOUDINIS_AVAILABLE) else 'Yes'}")
print("=" * 60)

In [ ]:
# 🔍 Grover's Algorithm Implementation
def create_grover_circuit(n_qubits, target_state):
    """Create a Grover circuit for searching a target state"""
    # Create quantum circuit
    qreg = QuantumRegister(n_qubits, 'q')
    creg = ClassicalRegister(n_qubits, 'c')
    circuit = QuantumCircuit(qreg, creg)
    
    # Initialize superposition
    circuit.h(qreg)
    
    # Oracle for target state
    oracle = QuantumCircuit(n_qubits)
    # Flip phase for target state (simplified for demonstration)
    for i, bit in enumerate(format(target_state, f'0{n_qubits}b')):
        if bit == '0':
            oracle.x(i)
    oracle.mcrz(np.pi, list(range(n_qubits-1)), n_qubits-1)
    for i, bit in enumerate(format(target_state, f'0{n_qubits}b')):
        if bit == '0':
            oracle.x(i)
    
    # Diffusion operator
    diffusion = QuantumCircuit(n_qubits)
    diffusion.h(range(n_qubits))
    diffusion.x(range(n_qubits))
    diffusion.mcrz(np.pi, list(range(n_qubits-1)), n_qubits-1)
    diffusion.x(range(n_qubits))
    diffusion.h(range(n_qubits))
    
    # Apply Grover iterations
    optimal_iterations = int(np.pi * np.sqrt(2**n_qubits) / 4)
    print(f"🔄 Applying {optimal_iterations} Grover iterations...")
    
    for _ in range(optimal_iterations):
        circuit.compose(oracle, inplace=True)
        circuit.compose(diffusion, inplace=True)
    
    # Measurement
    circuit.measure(qreg, creg)
    return circuit

# Initialize backend
try:
    backend_manager = QuantumBackendManager()
    backend_manager.select_backend(prefer_real=False)
    print("✅ Quantum backend initialized")
except:
    print("⚠️ Using classical simulation fallback")

# Demonstrate Grover search for a 4-bit key
print("\n🎯 Demonstrating Grover's algorithm for 4-bit key search...")
n_qubits = 4
secret_key = 0b1010  # Target key: 10 (binary: 1010)

print(f"🔑 Secret key: {secret_key} (binary: {format(secret_key, '04b')})")
print(f"🔍 Search space: 2^{n_qubits} = {2**n_qubits} possibilities")
print(f"📊 Classical brute force: ~{2**n_qubits // 2} attempts expected")
print(f"⚡ Grover's algorithm: ~{int(np.pi * np.sqrt(2**n_qubits) / 4)} iterations")

# Create Grover circuit
grover_circuit = create_grover_circuit(n_qubits, secret_key)

print(f"\n📊 Circuit characteristics:")
print(f"  • Circuit depth: {grover_circuit.depth()}")
print(f"  • Number of gates: {grover_circuit.count_ops()}")

In [ ]:
# 🧪 Execute Grover's Algorithm
print("🧪 Executing Grover's algorithm...")

try:
    if 'backend_manager' in locals():
        backend = backend_manager.get_backend()
        transpiled_circuit = transpile(grover_circuit, backend)
        job = backend.run(transpiled_circuit, shots=1024)
        result = job.result()
        counts = result.get_counts(transpiled_circuit)
        
        print("\n📈 Measurement results:")
        for state, count in sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]:
            probability = count / 1024
            binary_state = state[::-1]  # Reverse for proper bit order
            decimal_state = int(binary_state, 2)
            print(f"  State {decimal_state} ({binary_state}): {count}/1024 ({probability:.3f})")
        
        # Check if target was found
        target_binary = format(secret_key, '04b')
        success_rate = 0
        for state, count in counts.items():
            if state[::-1] == target_binary:
                success_rate = count / 1024
                break
        
        if success_rate > 0:
            print(f"\n✅ Target key found with probability: {success_rate:.3f}")
            print(f"🎯 Quantum advantage: {(1/(2**n_qubits)) / success_rate:.1f}x improvement over random search")
        else:
            print(f"\n❌ Target key not found in top results")
        
        # Store results for visualization
        grover_results = counts
        
except Exception as e:
    print(f"❌ Execution error: {e}")
    # Create mock results for visualization
    grover_results = {
        format(secret_key, '04b')[::-1]: 850,
        '0000': 50,
        '0001': 30,
        '1111': 40,
        '1000': 54
    }
    success_rate = 0.83

In [ ]:
# 📊 Grover's Algorithm Analysis and Visualization
print("📊 Analyzing Grover's algorithm performance...")

# Theoretical analysis for different key sizes
key_sizes = [4, 8, 16, 32, 64, 128, 256]
classical_complexity = [2**(size-1) for size in key_sizes]  # Average case
grover_complexity = [int(np.pi * np.sqrt(2**size) / 4) for size in key_sizes]
quantum_speedup = [classical_complexity[i] / grover_complexity[i] for i in range(len(key_sizes))]

# Create comprehensive visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Grover search results
if 'grover_results' in locals():
    states = [int(state[::-1], 2) for state in grover_results.keys()]
    probs = [count/1024 for count in grover_results.values()]
    
    bars = ax1.bar(states, probs, alpha=0.7, color='cyan')
    ax1.axvline(x=secret_key, color='red', linestyle='--', 
               label=f'Target key ({secret_key})', linewidth=2)
    
    # Highlight the target
    target_prob = 0
    for i, state in enumerate(states):
        if state == secret_key:
            bars[i].set_color('red')
            bars[i].set_alpha(0.9)
            target_prob = probs[i]
    
    ax1.set_xlabel('Key Value')
    ax1.set_ylabel('Probability')
    ax1.set_title('Grover Algorithm Key Search Results')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    if target_prob > 0:
        ax1.text(secret_key, target_prob + 0.05, f'{target_prob:.3f}', 
                ha='center', va='bottom', fontweight='bold', color='red')

# Complexity comparison
ax2.loglog(key_sizes, classical_complexity, 'b-o', label='Classical Brute Force', linewidth=2, markersize=8)
ax2.loglog(key_sizes, grover_complexity, 'r-s', label='Grover Algorithm', linewidth=2, markersize=8)
ax2.set_xlabel('Key Size (bits)')
ax2.set_ylabel('Search Complexity')
ax2.set_title('Search Complexity: Classical vs Grover')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Quantum speedup
ax3.semilogx(key_sizes, quantum_speedup, 'g-^', linewidth=2, markersize=8)
ax3.set_xlabel('Key Size (bits)')
ax3.set_ylabel('Quantum Speedup Factor')
ax3.set_title('Grover Quantum Speedup vs Key Size')
ax3.grid(True, alpha=0.3)

# Add annotations for key sizes
for i, (size, speedup) in enumerate(zip(key_sizes, quantum_speedup)):
    if size in [64, 128, 256]:
        ax3.annotate(f'{speedup:.0f}x', (size, speedup), 
                    xytext=(10, 10), textcoords='offset points',
                    fontweight='bold', color='green')

# Security impact timeline
years = np.arange(2020, 2045)
aes128_security = np.maximum(0, 128 - (years - 2020) * 2)  # Gradual reduction
aes256_security = np.maximum(0, 256 - (years - 2020) * 1.5)  # Slower reduction
classical_threshold = 80  # Minimum acceptable security level

ax4.plot(years, aes128_security, 'orange', linewidth=3, label='AES-128 Effective Security')
ax4.plot(years, aes256_security, 'blue', linewidth=3, label='AES-256 Effective Security')
ax4.axhline(y=classical_threshold, color='red', linestyle='--', 
           label=f'Security Threshold ({classical_threshold} bits)', linewidth=2)

# Fill areas
ax4.fill_between(years, aes128_security, alpha=0.2, color='orange')
ax4.fill_between(years, aes256_security, alpha=0.2, color='blue')

# Mark critical years
aes128_critical = years[aes128_security <= classical_threshold]
aes256_critical = years[aes256_security <= classical_threshold]

if len(aes128_critical) > 0:
    ax4.axvline(x=aes128_critical[0], color='orange', linestyle=':', alpha=0.7)
    ax4.text(aes128_critical[0], 50, f'AES-128\nCompromised\n{aes128_critical[0]}', 
            ha='center', va='center', bbox=dict(boxstyle='round', facecolor='orange', alpha=0.7))

if len(aes256_critical) > 0:
    ax4.axvline(x=aes256_critical[0], color='blue', linestyle=':', alpha=0.7)
    ax4.text(aes256_critical[0], 150, f'AES-256\nCompromised\n{aes256_critical[0]}', 
            ha='center', va='center', bbox=dict(boxstyle='round', facecolor='blue', alpha=0.7))

ax4.set_xlabel('Year')
ax4.set_ylabel('Effective Security Level (bits)')
ax4.set_title('AES Security Evolution vs Quantum Threat')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.set_ylim(0, 280)

plt.tight_layout()
plt.show()

print("\n📊 Grover's Algorithm Summary:")
print("  • Quadratic speedup over classical brute force")
print("  • Reduces effective key length by half")
print("  • Universal application to any search problem")
print("  • Threatens symmetric cryptography security levels")

print(f"\n🎯 Quantum Speedup Analysis:")
for i, size in enumerate([64, 128, 256]):
    if i < len(quantum_speedup):
        idx = key_sizes.index(size)
        print(f"  • {size}-bit keys: {quantum_speedup[idx]:.0f}x speedup")
        print(f"    Classical: ~{classical_complexity[idx]:e} operations")
        print(f"    Grover: ~{grover_complexity[idx]:e} operations")

In [ ]:
# 🔐 Password Cracking Simulation
print("\n🔐 Grover-Enhanced Password Cracking Simulation")

def simulate_password_attack(password_length, charset_size, target_hash):
    """Simulate password cracking with Grover's algorithm"""
    total_passwords = charset_size ** password_length
    classical_attempts = total_passwords // 2  # Average case
    grover_iterations = int(np.pi * np.sqrt(total_passwords) / 4)
    
    return {
        'total_space': total_passwords,
        'classical_attempts': classical_attempts,
        'grover_iterations': grover_iterations,
        'speedup': classical_attempts / grover_iterations
    }

# Password attack scenarios
password_scenarios = [
    {'name': '4-digit PIN', 'length': 4, 'charset': 10},
    {'name': '6-char lowercase', 'length': 6, 'charset': 26},
    {'name': '8-char alphanumeric', 'length': 8, 'charset': 62},
    {'name': '10-char full ASCII', 'length': 10, 'charset': 95},
    {'name': '12-char complex', 'length': 12, 'charset': 95}
]

print("\n📊 Password Cracking Analysis:")
attack_results = []

for scenario in password_scenarios:
    result = simulate_password_attack(
        scenario['length'], 
        scenario['charset'], 
        'dummy_hash'
    )
    
    attack_results.append({
        'scenario': scenario['name'],
        'total_space': result['total_space'],
        'classical_time': result['classical_attempts'],
        'grover_time': result['grover_iterations'],
        'speedup': result['speedup']
    })
    
    print(f"\n  {scenario['name']}:")
    print(f"    • Total passwords: {result['total_space']:e}")
    print(f"    • Classical attempts: {result['classical_attempts']:e}")
    print(f"    • Grover iterations: {result['grover_iterations']:e}")
    print(f"    • Quantum speedup: {result['speedup']:.1f}x")

# Visualize password cracking results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

scenarios = [r['scenario'] for r in attack_results]
classical_times = [r['classical_time'] for r in attack_results]
grover_times = [r['grover_time'] for r in attack_results]
speedups = [r['speedup'] for r in attack_results]

# Attack time comparison
x = np.arange(len(scenarios))
width = 0.35

ax1.bar(x - width/2, np.log10(classical_times), width, label='Classical', alpha=0.7, color='blue')
ax1.bar(x + width/2, np.log10(grover_times), width, label='Grover', alpha=0.7, color='red')

ax1.set_xlabel('Password Type')
ax1.set_ylabel('Attack Time (log₁₀ operations)')
ax1.set_title('Password Cracking: Classical vs Grover')
ax1.set_xticks(x)
ax1.set_xticklabels(scenarios, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Quantum speedup
bars = ax2.bar(scenarios, speedups, alpha=0.7, color='green')
ax2.set_xlabel('Password Type')
ax2.set_ylabel('Quantum Speedup Factor')
ax2.set_title('Grover Speedup for Password Cracking')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Add speedup labels
for bar, speedup in zip(bars, speedups):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + speedup*0.02, 
            f'{speedup:.1f}x', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# 🎯 Hash Function Preimage Attack Simulation
print("\n🎯 Hash Function Preimage Attack with Grover's Algorithm")

def simulate_hash_preimage_attack(hash_bits, target_hash="dummy"):
    """Simulate finding preimage for hash function using Grover"""
    search_space = 2 ** hash_bits
    classical_attempts = search_space // 2
    grover_iterations = int(np.pi * np.sqrt(search_space) / 4)
    
    # Estimate quantum resources needed
    logical_qubits = hash_bits
    gate_count = grover_iterations * hash_bits * 10  # Rough estimate
    
    return {
        'hash_bits': hash_bits,
        'search_space': search_space,
        'classical_attempts': classical_attempts,
        'grover_iterations': grover_iterations,
        'logical_qubits': logical_qubits,
        'gate_count': gate_count,
        'speedup': classical_attempts / grover_iterations
    }

# Hash function scenarios
hash_functions = [
    {'name': 'MD5', 'bits': 128},
    {'name': 'SHA-1', 'bits': 160},
    {'name': 'SHA-256', 'bits': 256},
    {'name': 'SHA-384', 'bits': 384},
    {'name': 'SHA-512', 'bits': 512}
]

print("\n📊 Hash Preimage Attack Analysis:")
hash_results = []

for hash_func in hash_functions:
    result = simulate_hash_preimage_attack(hash_func['bits'])
    hash_results.append({
        'name': hash_func['name'],
        'bits': hash_func['bits'],
        'classical_security': hash_func['bits'],
        'quantum_security': hash_func['bits'] // 2,  # Grover halves security
        'speedup': result['speedup'],
        'logical_qubits': result['logical_qubits'],
        'grover_iterations': result['grover_iterations']
    })
    
    print(f"\n  {hash_func['name']} ({hash_func['bits']}-bit):")
    print(f"    • Classical security: {hash_func['bits']} bits")
    print(f"    • Quantum security: {hash_func['bits'] // 2} bits")
    print(f"    • Grover iterations: {result['grover_iterations']:e}")
    print(f"    • Logical qubits needed: {result['logical_qubits']}")
    print(f"    • Quantum speedup: {result['speedup']:.1f}x")

# Visualize hash security analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

hash_names = [r['name'] for r in hash_results]
classical_sec = [r['classical_security'] for r in hash_results]
quantum_sec = [r['quantum_security'] for r in hash_results]
qubits_needed = [r['logical_qubits'] for r in hash_results]
grover_iters = [r['grover_iterations'] for r in hash_results]

# Security comparison
x = np.arange(len(hash_names))
width = 0.35

ax1.bar(x - width/2, classical_sec, width, label='Classical Security', alpha=0.7, color='blue')
ax1.bar(x + width/2, quantum_sec, width, label='Quantum Security', alpha=0.7, color='red')

ax1.set_xlabel('Hash Function')
ax1.set_ylabel('Security Level (bits)')
ax1.set_title('Hash Function Security: Classical vs Quantum')
ax1.set_xticks(x)
ax1.set_xticklabels(hash_names)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Quantum resources needed
ax2.semilogy(hash_names, grover_iters, 'go-', linewidth=2, markersize=8)
ax2.set_xlabel('Hash Function')
ax2.set_ylabel('Grover Iterations Needed')
ax2.set_title('Quantum Resources for Hash Preimage Attack')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Security timeline
years = np.arange(2020, 2050)
quantum_capability = np.minimum(512, 2 ** ((years - 2020) * 0.2))  # Exponential quantum growth

colors = ['red', 'orange', 'blue', 'green', 'purple']
for i, (name, bits) in enumerate(zip(hash_names, quantum_sec)):
    ax3.axhline(y=bits, color=colors[i], linestyle='--', label=f'{name} Quantum Security', alpha=0.7)

ax3.plot(years, quantum_capability, 'k-', linewidth=3, label='Quantum Capability')
ax3.fill_between(years, quantum_capability, alpha=0.2, color='black')
ax3.set_xlabel('Year')
ax3.set_ylabel('Effective Security Level (bits)')
ax3.set_title('Hash Function Quantum Threat Timeline')
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 300)

# Impact assessment
impact_categories = ['Digital Signatures', 'Password Hashing', 'Blockchain', 'TLS/SSL', 'File Integrity']
impact_scores = [9, 8, 10, 7, 6]  # Relative impact scores
colors_impact = ['red' if score >= 8 else 'orange' if score >= 6 else 'yellow' for score in impact_scores]

bars = ax4.barh(impact_categories, impact_scores, color=colors_impact, alpha=0.7)
ax4.set_xlabel('Impact Score (1-10)')
ax4.set_title('Grover Attack Impact by Application')
ax4.grid(True, alpha=0.3)

# Add score labels
for bar, score in zip(bars, impact_scores):
    ax4.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
            f'{score}', ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ Critical Hash Function Vulnerabilities:")
for result in hash_results:
    if result['quantum_security'] <= 128:
        print(f"  • {result['name']}: Reduced to {result['quantum_security']}-bit security")
        if result['quantum_security'] <= 64:
            print(f"    ⚠️ CRITICAL: May be breakable with near-term quantum computers")

## 🎯 Grover's Algorithm Attack Summary

This notebook demonstrated:

- ✅ **Grover's Algorithm**: Quantum search providing quadratic speedup
- ✅ **Key Recovery**: Accelerated brute-force attacks on symmetric encryption
- ✅ **Password Cracking**: Enhanced password attack capabilities
- ✅ **Hash Preimage**: Quantum attacks on cryptographic hash functions

### 🚨 **Critical Impact:**
- **Symmetric Keys**: Effective security halved (256-bit → 128-bit)
- **Hash Functions**: Preimage resistance significantly reduced
- **Password Security**: Complex passwords become more vulnerable
- **Digital Signatures**: Hash-based signatures weakened

### 📊 **Quantum Advantage:**
- **Search Speedup**: Quadratic improvement over classical brute-force
- **Universal Application**: Works on any unstructured search problem
- **Practical Threat**: Requires fewer qubits than Shor's algorithm
- **Near-term Risk**: Achievable with smaller quantum computers

### 🛡️ **Defensive Measures:**
- **Double Key Sizes**: Use 256-bit keys instead of 128-bit
- **Stronger Hash Functions**: Consider longer hash outputs
- **Enhanced Passwords**: Increase entropy and complexity
- **Quantum-Safe Protocols**: Migrate to post-quantum alternatives

---
**📧 Contact:** mauro.risonho@gmail.com  
**🌐 Project:** [Houdinis Framework](https://github.com/firebitsbr/Houdinis)  
**📜 License:** MIT - Use responsibly and ethically